In [ ]:
import pyodbc
import pandas as pd
# import numpy as np
# import plotly
# import plotly.graph_objects as go
# import plotly.express as px
# import math
# import statistics
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cnxn_str = ("Driver={SQL Server};"
            "Server=geosqle07;"
            "Database=MWD_Proemion;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)

In [ ]:
#cursor = cnxn.cursor()

In [ ]:
testquery = ("SELECT [TimeseiresId], [time], [value], t.[MesurementId], m.signal, [HumanTime] "
             "FROM [MWD_Proemion].[dbo].[Timeseries] t "
             "inner join [MWD_Proemion].[dbo].[Mesurements] m "
             "on t.mesurementid = m.mesurementid "
             "where HumanTime > '2023-01-01' and HumanTime < '2023-01-03' "
             "and (signal = 'value.Depth' "
             "or signal = 'value.Speed' or signal = 'value.Torque')"
             )

In [ ]:
#cursor.execute(testquery)

In [ ]:
#cnxn.close()

In [ ]:
data = pd.read_sql(testquery,cnxn)
data

In [ ]:
cnxn.close()

In [ ]:
speed_df = data[data['signal'] == 'value.Speed']
speed_df

In [ ]:
depth_df = data[data['signal'] == 'value.Depth']
depth_df

In [ ]:
torque_df = data[data['signal'] == 'value.Torque']
torque_df

In [ ]:
speed_df.rename(columns={'value': 'Speed'}, inplace=True)
depth_df.rename(columns={'value': 'Depth'}, inplace=True)
torque_df.rename(columns={'value': 'Torque'}, inplace=True)

In [ ]:
speed_df.drop(['TimeseiresId', 'MesurementId', 'signal'], axis=1, inplace= True)
speed_df

In [ ]:
depth_df.drop(['TimeseiresId', 'MesurementId', 'signal'], axis=1, inplace= True)
depth_df

In [ ]:
torque_df.drop(['TimeseiresId', 'MesurementId', 'signal'], axis=1, inplace= True)
torque_df

In [ ]:
mydf=speed_df.merge(depth_df, how='inner', on=['time', 'HumanTime']).merge(torque_df, how='inner', on=['time', 'HumanTime'])

In [ ]:
mydf.dtypes

In [ ]:
#Plot
xpoints = np.array(mydf['Speed'])
ypoints = np.array(mydf['HumanTime'])

plt.plot(xpoints, ypoints)
plt.show()

In [ ]:
#make subplots, plot side by side, raw vs corrected
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

xdata = mydf['Torque']
ydata = mydf['HumanTime']
xdepth = mydf['Depth']

#xtime = time_series['KSPSpeed']
#ytime = -time_series['DrillingDepth']

#y = np.arange(round(np.min(np.array(profile2['DrillingDepth'])))-10,1) #range(0,round(np.min(np.array(profile2['DrillingDepth'])))-10)
#x = np.arange(0,round(np.max(np.array(profile2['KSPSpeed'].dropna())))+11, 50)  # range(0,round(np.min(np.array(profile2['KSPSpeed'])))+10)

# yrange = [round(np.min(np.array(profile2['DrillingDepth'])))-2,-32]
# #xrange = [round(np.min(np.array(profile2['KSPSpeed'])))-10,round(np.max(np.array(profile2['KSPSpeed'])))+10]
# xrange = [400,round(np.max(np.array(profile2['KSPSpeed'].dropna())))+10]
# xrange1 = [-10,round(np.max(np.array(profile2['KSPSpeed'].dropna())))+10]

# print(x,y)


# fig.add_trace(go.Scatter(
#     x=xtime, 
#     y=ytime,
#     line_color='red',
#     name='Raw data'
#     ),
#     row=1, col=1)
fig.add_trace(go.Scatter(
    x=xdata, 
    y=ydata,
    line_color='blue',
    name='Corrected data'
    ),
    row=1, col=1)

fig.add_trace(go.Scatter(
    x=xdepth, 
    y=ydata,
    line_color='blue',
    name='Corrected data'
    ),
    row=1, col=2)

xaxis1=dict(
        #range = xrange1,
        zeroline=True,
        zerolinecolor = 'black',
        showline=True,
        showgrid=True,
        gridcolor = 'black',
        showticklabels=True,
        linecolor='black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),

xaxis2=dict(
    title='Depth',
    #range = xrange,
    zeroline=True,
    zerolinecolor = 'black',
    showline=True,
    showgrid=True,
    gridcolor = 'black',
    showticklabels=True,
    linecolor='black',
    linewidth=2,
    ticks='outside',
    tickfont=dict(
        family='Arial',
        size=12,
        color='rgb(82, 82, 82)',
        )
    )
yaxis=dict(
    #range = yrange, 
    showgrid=True,
    gridcolor = 'black',
    zeroline=True,
    zerolinecolor = 'black',
    linecolor = 'black',
    showline=True,
    showticklabels=True
    )

#fig.update_xaxes(title='KSPSpeed')
fig.update_xaxes(title='Torque',range=[-10, 500],zeroline=True, zerolinecolor = 'black',showline=True,
    showgrid=True,gridcolor = 'black', showticklabels=True,
    linecolor='black',
    linewidth=2,
    ticks='outside',row=1, col=1)
fig.update_yaxes(yaxis, title_text="Time",row=1, col=1)
fig.update_xaxes(xaxis2, row=1, col=2) #(range=[400,500], row=1, col=2)
fig.update_yaxes(yaxis, row=1, col=2)

fig.update_layout(
    xaxis2=dict(
        #range = xrange,
        zeroline=True,
        zerolinecolor = 'black',
        showline=True,
        showgrid=True,
        gridcolor = 'black',
        showticklabels=True,
        linecolor='black',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
            ),
        ),
    autosize=True,
    # margin=dict(
    #     autoexpand=False,
    #     l=50,
    #     r=50,
    #     t=110,
    # ),
    showlegend=False,
    plot_bgcolor='white',
    #width = 500,
    #height = 1000,
    title = 'T/D',
    #xaxis_title = 'KSPSpeed',
    #yaxis_title = 'DrillingDepth'
)

fig.show()